In [3]:
'''Question 3 -- code subparts'''

from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_boston
np.random.seed(0)

# load boston housing prices dataset
boston = load_boston()
x = boston['data']
N = x.shape[0]
x = np.concatenate((np.ones((506,1)),x),axis=1) #add constant one feature - no bias needed
d = x.shape[1]
y = boston['target']

idx = np.random.permutation(range(N))

/Users/costa/.local/share/virtualenvs/hw2work-W6V6Pwk9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this case special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative data

In [11]:
#helper function
def l2(A, B):
    '''
    Input: A is a Nxd matrix
           B is a Mxd matirx
    Output: dist is a NxM matrix where dist[i,j] is the square norm between A[i,:] and B[j,:]
    i.e. dist[i,j] = ||A[i,:]-B[j,:]||^2
    '''
    A_norm = (A**2).sum(axis=1).reshape(A.shape[0],1)
    B_norm = (B**2).sum(axis=1).reshape(1,B.shape[0])
    dist = A_norm+B_norm-2*A.dot(B.transpose())
    return dist

In [10]:
def calc_matrix_A(test_datum, x_train, tau):
    '''to calculate matrix A as needed with given formula in Q3 description'''
    
    test_datum = test_datum.reshape(test_datum.shape[0], 1).T

    # Nx1 array of squared norms
    norms = l2(x_train, test_datum2)
    scaled_norms = -1 * norms/(2*(tau**2))
    b = np.amax(scaled_norms) # for stability as shown in question description
    
    numerator = np.exp(scaled_norms-b) 
    denominator = np.exp(logsumexp(scaled_norms-b))

    # Create A - N_x_train x N_x_train matrix of zeros
    A = np.zeros((x_train.shape[0], x_train.shape[0]))

    # Calculate and set diagonals of A
    for i in range(0, x_train.shape[0]): 
        A[i][i] = numerator[i] / denominator 

    return A

In [12]:
#to implement
def LRLS(test_datum, x_train, y_train, tau, lam=1e-5):
    '''
    Given a test datum, it returns its prediction based on locally weighted regression

    Input: test_datum is a dx1 test vector
           x_train is the N_train x d design matrix
           y_train is the N_train x 1 targets vector
           tau is the local reweighting parameter
           lam is the regularization parameter
    output is y_hat the prediction on test_datum
    '''
    
    A = calc_matrix_A(test_datum, x_train, tau)
    
    # Num of rows and columns in x_train
    N_x_train = x_train.shape[0]
    D_x_train = x_train.shape[1]

    # identity matrix  -------after question edit by TA
    I = np.identity(D_x_train)
    
    # X, y for making sure we align visually with the formula in question description
    X = x_train
    y = y_train
    
    # perform matrix calculations from given closed form solution 
    P = np.matmul(np.matmul(X.T,A), X) - I*lam  # -------after question edit by TA
    Q = np.matmul(np.matmul(X.T, A), y)
    # numpy linear solver 
    weights_best = np.linalg.solve(P, Q)

    y_pred = np.matmul(test_datum.T, weights_best)

    return y_pred
    